In [ ]:
!pip install tiktoken
!pip install langchain
!pip install openai
!pip install PyPDF2

ERROR: /content/drive/MyDrive/qdrant-client-master is not a valid editable requirement. It should either be a path to a local project or a VCS URL (beginning with bzr+http, bzr+https, bzr+ssh, bzr+sftp, bzr+ftp, bzr+lp, bzr+file, git+http, git+https, git+ssh, git+git, git+file, hg+file, hg+http, hg+https, hg+ssh, hg+static-http, svn+ssh, svn+http, svn+https, svn+svn, svn+file).


In [ ]:
!pip install qdrant_client

In [ ]:
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Qdrant
from langchain.embeddings import OpenAIEmbeddings
from qdrant_client import QdrantClient,models
from qdrant_client.http.models import PointStruct
import os
import openai
import uuid

openai.api_key = ""

In [ ]:
# #create new cluseter in qdrant
record=0



connection = QdrantClient(
    url="",
    api_key="",
)

connection.recreate_collection(
    collection_name="chatbot_with_qdrant_shah",
    vectors_config=models.VectorParams(size=1536, distance=models.Distance.COSINE),
)
print("Create collection reponse:", connection)

info = connection.get_collection(collection_name="chatbot_with_qdrant_shah")

print("Collection info:", info)
for get_info in info:
  print(get_info)

Create collection reponse: <qdrant_client.qdrant_client.QdrantClient object at 0x7f6d33fa2b30>
Collection info: status=<CollectionStatus.GREEN: 'green'> optimizer_status=<OptimizersStatusOneOf.OK: 'ok'> vectors_count=0 indexed_vectors_count=0 points_count=0 segments_count=2 config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=1536, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None), shard_number=1, replication_factor=1, write_consistency_factor=1, on_disk_payload=True), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=1), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), quantization_config=None) pa

In [ ]:
# #funcrion for read data from pdf
def read_data_from_pdf():
    pdf_path = '/content/greedy_dog.pdf'
    text = ""  # for storing the extracted text

    with open(pdf_path, 'rb') as file:
        pdf_reader = PdfReader(file)

        for page in pdf_reader.pages:
            text += page.extract_text()

    return text

In [ ]:
#code for making text into chunks

def get_text_chunks(text):
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )
    chunks = text_splitter.split_text(text)
    return chunks

In [ ]:
# #code for convert chunks into embeddings
def get_embedding(text_chunks, model_id="text-embedding-ada-002"):
    points = []
    for idx, chunk in enumerate(text_chunks):
        response = openai.Embedding.create(
            input=chunk,
            model=model_id
        )
        embeddings = response['data'][0]['embedding']
        point_id = str(uuid.uuid4())  # Generate a unique ID for the point

        points.append(PointStruct(id=point_id, vector=embeddings, payload={"text": chunk}))

    return points




In [ ]:
# # code for insert data into qdrant database

def insert_data(get_points):
    operation_info = connection.upsert(
    collection_name="chatbot_with_qdrant_shah",
    wait=True,
    points=get_points
)



In [ ]:
# code for searching
def create_answer_with_context(query):
    response = openai.Embedding.create(
        input=query,

        model="text-embedding-ada-002"
    )
    embeddings = response['data'][0]['embedding']

    search_result = connection.search(
        collection_name="chatbot_with_qdrant_shah",
        query_vector=embeddings,
        limit=1
    )

    prompt = "Context:\n"
    for result in search_result:
        prompt += result.payload['text'] + "\n---\n"
    prompt += "Question:" + query + "\n---\n" + "Answer:"

    print("----PROMPT START----")
    print(":", prompt)
    print("----PROMPT END----")

    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt}
        ]
        )

    return completion.choices[0].message.content


In [ ]:
def main():
  get_raw_text=read_data_from_pdf()
  chunks=get_text_chunks(get_raw_text)
  vectors=get_embedding(chunks)

  insert_data(vectors)
  question="what did the dog see at the bakery?"
  answer=create_answer_with_context(question)
  print(answer)


In [ ]:
if __name__ == '__main__':
    main()


----PROMPT START----
: Context:
cast his gaze around and discovered a shelf stacked with freshly baked blueberry pies. The scent of 
the juicy berries tickled his nose, and without thinking twice, he jumped onto the shelf, knocking 
down several pies in the process. Unfazed, Max greedily w olfed down one pie after another, savoring 
the burst of flavors.  
 
Little did Max know, his noisy feast hadn't gone unnoticed. The bakery's owner, Mrs. Thompson, had 
returned from her brief break and was shocked to find her beloved pies in disarray. She fol lowed the 
trail of crumbs, which led her straight to the mischievous Max perched atop the shelf, licking his lips 
guiltily.  
 
Mrs. Thompson couldn't help but smile at the sight of the rascally dog. Though she was upset about 
her pies, she knew Max had acted  out of instinct. Max, sensing his mischief had been discovered, 
wagged his tail sheepishly, as if to apologize.
---
Question:what did the dog see at the bakery?
---
Answer:
----PROMP